In [1]:
import argparse
import json
import logging
import sys

import copy
import numpy as np
import pandas as pd
import pdb

import glob
import os
from collections import defaultdict
import seaborn as sns
import matplotlib.pyplot as plt

import loki
from loki.util import constraints, postprocessing
from loki.solver import sat
from loki.util.postprocessing import *

%load_ext autoreload
%autoreload 2

In [2]:
## add progress bar showing upload to postgres
## print out some model variables while sat solving
## remove error messages

# Setting up parameters

In [3]:

os.chdir("/flash1/pari/loki")
parser = argparse.ArgumentParser(description='Run Loki.')
parser.add_argument('--config', '-c', default='application.conf')
parser.add_argument('--workload', '-w', default='imdb')
parser.add_argument('--db',  default='imdb')
parser.add_argument('--table', '-t', default='n')
parser.add_argument('--vars_per_col', '-n', default=1000)
parser.add_argument('--logfile', '-lf', default='logs/loki.log')
parser.add_argument('--verbose', '-v', default=False, action='store_true')
#args = parser.parse_args()
args = parser.parse_args(args=[])

# Input Files (IR of Parsed Query Workload)

In [4]:
loki.load_config(args.config)
c_df = pd.read_csv(loki.config[args.workload]['constraints_df'])
c_df = c_df.sample(frac=1.0)
c_df.head(5)

,exprhash,RowCount,InputCardinality,RowSql,Column,input,jobid,Op,Value,Selectivity
37578,556653557154222418862973223862447744203226576103,782326,14835720,SELECT COUNT(*) from movie_info AS mi2 WHERE m...,mi2.info,mi,1459,=,'English',0.052733
6625,280863473239219469668760861684563098362757236652,1,134170,SELECT COUNT(*) from keyword AS k WHERE k.keyw...,k.keyword,k,251,=,'interview',0.000007
36550,106002214293785573864802763455791885469084566965,88707,14835720,SELECT COUNT(*) from movie_info AS mi2 WHERE m...,mi2.info,mi,1416,=,'Spanish',0.005979
32312,1325596614819504186708354884550721622722117873135,1,7,SELECT COUNT(*) from kind_type AS kt WHERE kt....,kt.kind,kt,1236,=,'video game',0.142857
46465,608651404372439871429376739572497264345874208274,1,12,SELECT COUNT(*) from role_type AS rt WHERE rt....,rt.role,rt,1799,=,'cinematographer',0.083333


# Converting the cardinalities ---> SAT constraints, and solving it for a potential database that satisfies those constraints

In [6]:
import warnings
warnings.filterwarnings('ignore')
import time

start = time.time()
logging.basicConfig(
    level=logging.DEBUG if args.verbose else logging.INFO,
    format="%(asctime)s %(levelname)-8s [%(name)s]  %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    handlers=[logging.FileHandler(args.logfile), logging.StreamHandler()])

logger = logging.getLogger('stdout')

#logger.info('Loki initialized with configuration:\n%s' % json.dumps(loki.config, sort_keys=True, indent=4))

print('Loki initialized with configuration: \n{}'.format(json.dumps(loki.config, sort_keys=True, indent=4)))
df = pd.read_csv(loki.config[args.workload]['df'])
tables = set(df['input'])
table_columns = {table: set(df['column'].where(df['input'] == table).dropna()) for table in tables}

table = args.table
orig_table = table
# table = '"{}"'.format(table)

columns = table_columns[table]

constraints_df = constraints.get_constraints_df(c_df, table)
print("Operators used: ", set(constraints_df["Op"]))

table_cardinality = constraints.get_table_cardinality(constraints_df)
co_optimized_columns = constraints.get_co_optimized_columns(constraints_df, columns)
programs = constraints.get_programs(co_optimized_columns)

leftover_constraints = []

vars_per_col = args.vars_per_col

solutions = []

constraints_df = constraints_df[constraints_df["Value0"] != "None"]
constraints_df = constraints_df[constraints_df["Value1"] != "None"]
constraints_df = constraints_df[constraints_df["Value"] != "None"]
# pdb.set_trace()
for program in programs:
    #logger.info(f'Solving: {program}')
    print(f'Solving: {program}')
    constraints_ = constraints.parse_constraints(program, constraints_df)
    model, vars, cols, col_values_ids_map = sat.build_model(program, constraints_, leftover_constraints, table_cardinality, vars_per_col)
    solution = sat.solve(model, vars, cols, col_values_ids_map, vars_per_col)
    solutions.append(solution)
    #logger.info(f'Solved: {program}')
    print(f'Solved: {program}')

# Combine all programs' solutions into a single dictionary solution
full_solution = {k: v for s in solutions for k, v in s.items()}

postprocessing.apply_leftover_constraints(full_solution, leftover_constraints)
solution_df = postprocessing.solution_to_df(full_solution)
final_solution_df = postprocessing.scale_solution_df(solution_df, table_cardinality, vars_per_col)

final_solution_df.to_csv(f'results/{orig_table}.csv')

print("Generating solutions took: ", time.time()-start)

Loki initialized with configuration: 
{
    "imdb": {
        "constraints_df": "sample-data/literal_df.csv",
        "df": "sample-data/op_df.csv"
    }
}
Operators used:  {'=', 'like', '=|='}
Solving: {'name'}
Problem scale: 155000
Stopping after 70 constraints!
Status = OPTIMAL
Solved: {'name'}
Solving: {'surname_pcode', 'name_pcode_nf', 'gender', 'name_pcode_cf'}
Problem scale: 1194000
Stopping after 1012 constraints!
Status = OPTIMAL
Solved: {'surname_pcode', 'name_pcode_nf', 'gender', 'name_pcode_cf'}
Generating solutions took:  105.8959903717041


# Exploring the solution

In [7]:
## a lot of the cells in the final solution are unspecified since we may not have enough data
tmp = final_solution_df.dropna()
tmp.sample(frac=0.1).head(10)

,name,name_pcode_nf,gender,surname_pcode,name_pcode_cf
8226,%ra%,G5356,f,A436,R363
94912,%ke%,P3614,m,O165,S4153
174130,%se%,G6216,m,%l15%,H5435
11747,%wal%,G2316,m,%j1%,G5242
180267,%ra%,M6165,m,%m3%,R3626
57781,%sa%,M2453,f,O425,B4525
82319,%ma%,C6216,m,R242,T5212
197686,%min%,P3635,f,B62,P5215
84697,%en%,D5423,f,B452,S5236
195912,%min%,P3635,f,B62,P5215


# Uploading the created table to Postgres

In [11]:
DKIND="s1"

In [ ]:

## TO try: shuffle=False; null_strs=True
upload_to_postgres(final_solution_df, args.db, args.table, DKIND, shuffle=True, null_strs=False)

# Setting up execution result directories

In [8]:
import sys
import os
import shutil

os.chdir("/spinning/pari/WorkloadCharacterization")

try:
    shutil.rmtree('./new_results')
except Exception as e:
    pass

try:
    os.mkdir("./new_results")
except:
    pass

NUMQ = 100

# Executing Workload Queries on true data

In [9]:
CMD = "python3 eval_data.py --data_kind true_cols --num_queries {} \
--inp_to_eval n --port 5432 --workload ceb --db_name imdb --result_dir new_results".format(NUMQ)
out = os.system(CMD)

data/ceb-all/sqls/dfs/expr_df.csv
Number of sqls to evaluate: 100
SELECT COUNT(*) from n_true_cols AS n WHERE n.gender in ('m')
SELECT COUNT(*) from n_true_cols AS n WHERE n.gender in ('f','m') AND n.name_pcode_nf in ('C6231','E4213','F6362','F6525','J513','M6251','M6263','P3625','R1632','R1636','R2631','R2632','S2153')
SELECT COUNT(*) from n_true_cols AS n WHERE n.gender in ('f')
SELECT COUNT(*) from n_true_cols AS n WHERE n.gender in ('m') AND n.name_pcode_nf in ('A6361','D1326','D1352','G6262','J5241','J5245','J5263','L2142','M6216','S3151')
SELECT COUNT(*) from n_true_cols AS n WHERE n.gender in ('f','m')
SELECT COUNT(*) from n_true_cols AS n WHERE n.gender in ('f') AND n.surname_pcode in ('B2','B6','B62','B653','C2','C5','J52','J525','L15','L52','M62','P62','R3','W425')
SELECT COUNT(*) from n_true_cols AS n WHERE n.gender in ('f','m') AND n.name_pcode_nf in ('A4236','A5356','A5361','E3631','J25','M2412','M2415','M6242','M6352','P3656','R1632','S2525','S3541','V2362')
SELECT COUNT(

# Executing Workload Queries on our Generated Data

In [12]:
CMD2 = "python3 eval_data.py --data_kind {} --num_queries {} \
--inp_to_eval n --port 5432 --workload ceb --db_name imdb --result_dir new_results".format(DKIND, NUMQ)
out = os.system(CMD2)

data/ceb-all/sqls/dfs/expr_df.csv
Number of sqls to evaluate: 100
SELECT COUNT(*) from n_s1 AS n WHERE n.gender in ('m')
SELECT COUNT(*) from n_s1 AS n WHERE n.gender in ('f','m') AND n.name_pcode_nf in ('C6231','E4213','F6362','F6525','J513','M6251','M6263','P3625','R1632','R1636','R2631','R2632','S2153')
SELECT COUNT(*) from n_s1 AS n WHERE n.gender in ('f')
SELECT COUNT(*) from n_s1 AS n WHERE n.gender in ('m') AND n.name_pcode_nf in ('A6361','D1326','D1352','G6262','J5241','J5245','J5263','L2142','M6216','S3151')
SELECT COUNT(*) from n_s1 AS n WHERE n.gender in ('f','m')
SELECT COUNT(*) from n_s1 AS n WHERE n.gender in ('f') AND n.surname_pcode in ('B2','B6','B62','B653','C2','C5','J52','J525','L15','L52','M62','P62','R3','W425')
SELECT COUNT(*) from n_s1 AS n WHERE n.gender in ('f','m') AND n.name_pcode_nf in ('A4236','A5356','A5361','E3631','J25','M2412','M2415','M6242','M6352','P3656','R1632','S2525','S3541','V2362')
SELECT COUNT(*) from n_s1 AS n WHERE n.gender in ('m') AND n.n

# Exploring the results

In [13]:
def extract_val(d, key):
    val_start = d.find(key)
    val_end = val_start + d[val_start:].find(",")
    if val_end < val_start:
        val_end = val_start + d[val_start:].find(")")
    val = d[val_start:val_end].split("=")[1]
    return val.replace("'", "")

def load_logs(logfile):
    with open(logfile, "r") as f:
        data = f.readlines()
    
    retdata = defaultdict(list)
    alg = None
    #port = None
    
    for d in data:
        if "data_kind" in d:
            #d = d.replace("\n", "")
            #d = d.replace(" ", "")
            data_kind = extract_val(d, "data_kind")
            inp_to_eval = extract_val(d, "inp_to_eval")
            nq = int(extract_val(d, "num_queries"))
            wk = extract_val(d, "workload")
            if "port" not in d:
                break
            
            port = int(extract_val(d, "port"))
            skip_likes = int(extract_val(d, "skip_likes"))
            
        if "-->" in d:
            linedata = d.split(" ")
            ehash = linedata[5]
            rt = float(linedata[-1].replace("\n", ""))
            #rep = int(linedata[-5])
            #qerr = float(linedata[-3].replace("QErr:", ""))
            
            estc = float(linedata[-3].replace("EstC:", ""))
            truec = float(linedata[-5].replace("TrueC:", ""))
            rep = int(linedata[-9])
            qerr = float(linedata[-7].replace("QErr:", ""))
            
            retdata["ehash"].append(ehash)
            retdata["rt"].append(rt)
            retdata["rep"].append(rep)
            retdata["data_kind"].append(str(data_kind))
            retdata["inp_to_eval"].append(inp_to_eval)
            retdata["num_queries"].append(nq)
            retdata["workload"].append(wk)
            retdata["port"].append(port)
            retdata["skip_likes"].append(skip_likes)
            retdata["qerr"].append(qerr)
            retdata["truec"].append(truec)
            retdata["estc"].append(estc)
            
    df = pd.DataFrame(retdata)
    return df

In [14]:
LOGDIR = "./new_results/"
fns = glob.glob(LOGDIR + "*.log")
dfs = []

for fn in fns:
    df = load_logs(fn)
    dfs.append(df)
df = pd.concat(dfs)

In [15]:
mapping = {}
mapping["true_cols"] = "TrueData"
mapping["gen_shuffle"] = "GeneratedData"
mapping["s1"] = "GeneratedData"
mapping["s2"] = "GeneratedData"

df["data_kind"] = df.apply(lambda x: mapping[x["data_kind"]] , axis=1)

In [16]:
tmp = df.groupby(["ehash", "data_kind"])["rt"].mean().reset_index()
edf = tmp.set_index(['ehash', 'data_kind'])['rt'].unstack().reset_index()

In [17]:
import pickle
esqls = {}
if os.path.exists("esqls.pkl"):
    with open('esqls.pkl', 'rb') as handle:
        esqls = pickle.load(handle)
    
eqerrs = {}
etruecs = {}
estcs = {}

qdf = df[df["data_kind"] == "GeneratedData"]
for i,row in qdf.iterrows():
    eqerrs[row["ehash"]] = row["qerr"]
    etruecs[row["ehash"]] = row["truec"]
    estcs[row["ehash"]] = row["estc"]
    
true_rt = round(df.groupby(["data_kind"])["rt"].sum()["TrueData"], 2)
gen_rt = round(df.groupby(["data_kind"])["rt"].sum()["GeneratedData"], 2)

In [18]:
# fig, ax = plt.subplots(nrows=1,ncols=1,figsize=(6,5))
# true_rt = round(df.groupby(["data_kind"])["rt"].sum()["TrueData"], 2)
# gen_rt = round(df.groupby(["data_kind"])["rt"].sum()["GeneratedData"], 2)

# title = "Total Latency, True: {}; Generated: {}".format(true_rt, gen_rt)

# sns.scatterplot(data=edf, x = "GeneratedData", y="TrueData", alpha=1.0, ax=ax)

# ax.set_ylabel("True Data", fontsize=16)
# ax.set_xlabel("Generated Data", fontsize=16)

# plt.title(title, fontsize=16)

# #FN= "{}-{}-scatterplot.pdf".format(INP , NUMQ)
# #plt.savefig(FN, bbox_inches="tight")
# plt.show()

In [19]:
import sqlparse
edf["sql"] = edf.apply(lambda x: sqlparse.format(esqls[x["ehash"]], reindent=True,keyword_case="upper") ,
                       axis=1)
edf["qerr"] = edf.apply(lambda x: eqerrs[x["ehash"]] ,axis=1)
edf["estc"] = edf.apply(lambda x: estcs[x["ehash"]] ,axis=1)
edf["truec"] = edf.apply(lambda x: etruecs[x["ehash"]] ,axis=1)

sqlparse.format(edf.sql.values[5], reindent=True, keyword_case='upper')

"SELECT COUNT(*)\nFROM name AS n\nWHERE n.gender in ('f',\n                   'm')\n  AND n.surname_pcode in ('A436',\n                          'B2',\n                          'B452',\n                          'C16',\n                          'C462',\n                          'C65',\n                          'H2',\n                          'J52',\n                          'L2',\n                          'M5',\n                          'R2',\n                          'S23',\n                          'W425')"

In [20]:
from bokeh.io import output_file, show, reset_output, output_notebook
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure
import pandas as pd

reset_output()
output_notebook()
#output_file("name-final-results.html")

source = ColumnDataSource(data=edf)

# Define the hover tool to display the 'data_kind' column from edf
hover = HoverTool(tooltips=[
                             ("Gen Data", "@GeneratedData"),
                             ("True Data", "@TrueData"),
                             ("Gen Cardinality", "@estc"),
                             ("True Cardinality", "@truec"),
                             #("Cardinality Q-Error", "@qerr"),
                             ("SQL", "@sql"),
                            ])

# Create the Bokeh figure
p = figure(title="Total Latency, True: {}; Generated: {}".format(true_rt, gen_rt), 
                  tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
          width=600, height=500,
          )

# Add the scatter plot to the Bokeh figure
p.scatter('GeneratedData', 'TrueData', source=source, 
         size=8)
p.add_tools(hover)

# Set the x and y axis labels
p.xaxis.axis_label = "Generated Data"
p.yaxis.axis_label = "True Data"

# Display the plot
show(p)

Loading BokehJS ...

In [21]:
DKIND2="s2"

In [25]:
DKIND2="s2"

## TO try: shuffle=False; null_strs=True
upload_to_postgres(final_solution_df, args.db, args.table, DKIND, shuffle=False, null_strs=True)

done updating NULL values w/ random strings
uploading to postgres took:  342.73


In [26]:
import sys
import os
import shutil

os.chdir("/spinning/pari/WorkloadCharacterization")

try:
    shutil.rmtree('./new_results')
except Exception as e:
    pass

try:
    os.mkdir("./new_results")
except:
    pass

NUMQ = 100

In [27]:
CMD = "python3 eval_data.py --data_kind true_cols --num_queries {} \
--inp_to_eval n --port 5432 --workload ceb --db_name imdb --result_dir new_results".format(NUMQ)
out = os.system(CMD)

data/ceb-all/sqls/dfs/expr_df.csv
Number of sqls to evaluate: 100
SELECT COUNT(*) from n_true_cols AS n WHERE n.gender in ('m')
SELECT COUNT(*) from n_true_cols AS n WHERE n.gender in ('f','m') AND n.name_pcode_nf in ('C6231','E4213','F6362','F6525','J513','M6251','M6263','P3625','R1632','R1636','R2631','R2632','S2153')
SELECT COUNT(*) from n_true_cols AS n WHERE n.gender in ('f')
SELECT COUNT(*) from n_true_cols AS n WHERE n.gender in ('m') AND n.name_pcode_nf in ('A6361','D1326','D1352','G6262','J5241','J5245','J5263','L2142','M6216','S3151')
SELECT COUNT(*) from n_true_cols AS n WHERE n.gender in ('f','m')
SELECT COUNT(*) from n_true_cols AS n WHERE n.gender in ('f') AND n.surname_pcode in ('B2','B6','B62','B653','C2','C5','J52','J525','L15','L52','M62','P62','R3','W425')
SELECT COUNT(*) from n_true_cols AS n WHERE n.gender in ('f','m') AND n.name_pcode_nf in ('A4236','A5356','A5361','E3631','J25','M2412','M2415','M6242','M6352','P3656','R1632','S2525','S3541','V2362')
SELECT COUNT(

In [28]:
CMD2 = "python3 eval_data.py --data_kind {} --num_queries {} \
--inp_to_eval n --port 5432 --workload ceb --db_name imdb --result_dir new_results".format(DKIND2, NUMQ)
out = os.system(CMD2)

data/ceb-all/sqls/dfs/expr_df.csv
Number of sqls to evaluate: 100
SELECT COUNT(*) from n_s2 AS n WHERE n.gender in ('m')
SELECT COUNT(*) from n_s2 AS n WHERE n.gender in ('f','m') AND n.name_pcode_nf in ('C6231','E4213','F6362','F6525','J513','M6251','M6263','P3625','R1632','R1636','R2631','R2632','S2153')
SELECT COUNT(*) from n_s2 AS n WHERE n.gender in ('f')
SELECT COUNT(*) from n_s2 AS n WHERE n.gender in ('m') AND n.name_pcode_nf in ('A6361','D1326','D1352','G6262','J5241','J5245','J5263','L2142','M6216','S3151')
SELECT COUNT(*) from n_s2 AS n WHERE n.gender in ('f','m')
SELECT COUNT(*) from n_s2 AS n WHERE n.gender in ('f') AND n.surname_pcode in ('B2','B6','B62','B653','C2','C5','J52','J525','L15','L52','M62','P62','R3','W425')
SELECT COUNT(*) from n_s2 AS n WHERE n.gender in ('f','m') AND n.name_pcode_nf in ('A4236','A5356','A5361','E3631','J25','M2412','M2415','M6242','M6352','P3656','R1632','S2525','S3541','V2362')
SELECT COUNT(*) from n_s2 AS n WHERE n.gender in ('m') AND n.n

In [29]:
mapping = {}
mapping["true_cols"] = "TrueData"
mapping["gen_shuffle"] = "GeneratedData"
mapping["s1"] = "GeneratedData"
mapping["s2"] = "GeneratedData"

LOGDIR = "./new_results/"
fns = glob.glob(LOGDIR + "*.log")
dfs = []

for fn in fns:
    df = load_logs(fn)
    dfs.append(df)
df = pd.concat(dfs)
df["data_kind"] = df.apply(lambda x: mapping[x["data_kind"]] , axis=1)
tmp = df.groupby(["ehash", "data_kind"])["rt"].mean().reset_index()
edf = tmp.set_index(['ehash', 'data_kind'])['rt'].unstack().reset_index()

import pickle
esqls = {}
if os.path.exists("esqls.pkl"):
    with open('esqls.pkl', 'rb') as handle:
        esqls = pickle.load(handle)
    
eqerrs = {}
etruecs = {}
estcs = {}

qdf = df[df["data_kind"] == "GeneratedData"]
for i,row in qdf.iterrows():
    eqerrs[row["ehash"]] = row["qerr"]
    etruecs[row["ehash"]] = row["truec"]
    estcs[row["ehash"]] = row["estc"]
    
true_rt = round(df.groupby(["data_kind"])["rt"].sum()["TrueData"], 2)
gen_rt = round(df.groupby(["data_kind"])["rt"].sum()["GeneratedData"], 2)

import sqlparse
edf["sql"] = edf.apply(lambda x: sqlparse.format(esqls[x["ehash"]], reindent=True,keyword_case="upper") ,
                       axis=1)
edf["qerr"] = edf.apply(lambda x: eqerrs[x["ehash"]] ,axis=1)
edf["estc"] = edf.apply(lambda x: estcs[x["ehash"]] ,axis=1)
edf["truec"] = edf.apply(lambda x: etruecs[x["ehash"]] ,axis=1)

sqlparse.format(edf.sql.values[5], reindent=True, keyword_case='upper')

"SELECT COUNT(*)\nFROM name AS n\nWHERE n.gender in ('f',\n                   'm')\n  AND n.surname_pcode in ('A436',\n                          'B2',\n                          'B452',\n                          'C16',\n                          'C462',\n                          'C65',\n                          'H2',\n                          'J52',\n                          'L2',\n                          'M5',\n                          'R2',\n                          'S23',\n                          'W425')"

In [30]:
from bokeh.io import output_file, show, reset_output, output_notebook
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure
import pandas as pd

reset_output()
output_notebook()
#output_file("name-final-results.html")

source = ColumnDataSource(data=edf)

# Define the hover tool to display the 'data_kind' column from edf
hover = HoverTool(tooltips=[
                             ("Gen Data", "@GeneratedData"),
                             ("True Data", "@TrueData"),
                             ("Gen Cardinality", "@estc"),
                             ("True Cardinality", "@truec"),
                             #("Cardinality Q-Error", "@qerr"),
                             ("SQL", "@sql"),
                            ])

# Create the Bokeh figure
p = figure(title="Total Latency, True: {}; Generated: {}".format(true_rt, gen_rt), 
                  tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
          width=600, height=500,
          )

# Add the scatter plot to the Bokeh figure
p.scatter('GeneratedData', 'TrueData', source=source, 
         size=8)
p.add_tools(hover)

# Set the x and y axis labels
p.xaxis.axis_label = "Generated Data"
p.yaxis.axis_label = "True Data"

# Display the plot
show(p)

Loading BokehJS ...